In [3]:
from multiprocessing import shared_memory
import joblib
import os
import pandas as pd 
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
import shap
import pandas as pd
def get_global_expl_plot(conf_name):

    loaded_model = joblib.load('./results-removed-columns/models/RF/'+conf_name+'.pkl')

    df = pd.read_csv("./Data/processed/"+conf_name+".csv", index_col = 0)
    X =df.drop(['target', 'function', 'instance'], axis = 1)
    X = X.rename(columns=lambda s: s.split("/")[-1])
    y = df.iloc[:,-1]

    explainer = shap.TreeExplainer(loaded_model)
    shap_values = explainer.shap_values(X)
    print(shap_values.shape)

get_global_expl_plot("dim_5_budget_500_conf_0")

(120, 46)


In [69]:
from multiprocessing import shared_memory
import joblib
import os
import pandas as pd 
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
import shap
import pandas as pd

def get_global_shap(conf_name):
    loaded_model = joblib.load('./results/models/RF/'+conf_name+'.pkl')
    df = pd.read_csv("./Data/processed/"+conf_name+".csv", index_col = 0)
    X = df.drop(['target'], axis = 1)
    X = X.rename(columns=lambda s: s.split("/")[-1])
    y = df.iloc[:,-1]

    logo = LeaveOneGroupOut()
    groups = X['instance']
    logo.get_n_splits(X, y, groups)
    dt = []
    for train_index, test_index in logo.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_removed = X_train.drop(['instance', 'function'], axis=1)
        for instance in np.unique(np.array(X_train['instance'])):
            loaded_model = loaded_model.fit(X_train_removed, y_train)
            explainer = shap.TreeExplainer(loaded_model)
            data = X_train[X_train['instance']==instance]
            func = np.array(data['function']).reshape(24,1)
            ins = np.array(data['instance']).reshape(24,1)
            data = data.drop(['instance', 'function'], axis=1)
            shap_values = np.concatenate((func, ins, explainer.shap_values(data)), axis=1)
            dt.extend(shap_values)

    df_dt = pd.DataFrame(dt)

    final_shap = []
    for f in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 2, 3, 4, 5, 6, 7, 8, 9]:
        for i in range(1, 6):
            d = df_dt[(df_dt[0]==f) & (df_dt[1]==i)]
            d = d.drop([d.columns[0], d.columns[1]], axis=1)
            avg = np.average(d, axis = 0)
            final_shap.append(avg)
    return final_shap

get_global_shap("dim_5_budget_500_conf_0")

10 1
10 2
10 3
10 4
10 5
11 1
11 2
11 3
11 4
11 5
12 1
12 2
12 3
12 4
12 5
13 1
13 2
13 3
13 4
13 5
14 1
14 2
14 3
14 4
14 5
15 1
15 2
15 3
15 4
15 5
16 1
16 2
16 3
16 4
16 5
17 1
17 2
17 3
17 4
17 5
18 1
18 2
18 3
18 4
18 5
19 1
19 2
19 3
19 4
19 5
1 1
1 2
1 3
1 4
1 5
20 1
20 2
20 3
20 4
20 5
21 1
21 2
21 3
21 4
21 5
22 1
22 2
22 3
22 4
22 5
23 1
23 2
23 3
23 4
23 5
24 1
24 2
24 3
24 4
24 5
2 1
2 2
2 3
2 4
2 5
3 1
3 2
3 3
3 4
3 5
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
5 4
5 5
6 1
6 2
6 3
6 4
6 5
7 1
7 2
7 3
7 4
7 5
8 1
8 2
8 3
8 4
8 5
9 1
9 2
9 3
9 4
9 5
120


[array([-0.33500738,  0.07952129,  0.85305921, -0.52733915,  0.38481147,
         0.40321374,  0.0565727 ,  0.55686077,  0.21004626,  9.05391263,
         8.31246223, 12.35755397, -0.03523216,  0.08847165,  0.67454691,
        -2.56772727, -0.3880007 ,  0.35655792,  0.8608977 , -0.79404679,
         7.38220597, -0.88881715, -4.75492128,  5.54597236, -0.89844188,
         9.21763107,  4.35559191, -2.04544578,  0.54604777,  0.69279665,
         4.64150156,  2.45190256,  2.64291042, -0.07727476, -0.60943234,
        -2.10370245,  0.13796793,  0.27515293,  0.40594283, -0.1745048 ,
        -0.53497546,  0.08383231, -0.05593228,  0.72482647,  0.36275405,
        -0.06645423]),
 array([-2.32261527e-02,  4.90757236e-01,  1.01853236e+00, -5.39817064e-01,
         1.28325298e+00,  5.33701953e-01,  7.56136404e-01,  6.13852394e-01,
         1.53923958e+00,  8.77694930e+00,  2.16284738e+01,  1.73700823e+01,
         8.85722378e-01,  1.57539876e+00,  4.18307706e+00,  2.65856224e+00,
         1.41849